In [ ]:
ft_t <- as.data.frame(t(new_ft)) # transposing the new ft so that the rownames of ft_t and rownames of new_md will be the same
ft_t <- ft_t %>%
        mutate_all(as.numeric) %>% #converting all values to numeric
        mutate(filename=rownames(ft_t)) %>% #adding a column filename with the rownames of ft_t
        select(filename, everything()) #positioning filename as 1st column

In [ ]:
head(ft_t, n=3)

In [ ]:
identical(new_md$filename,ft_t$filename) #should return TRUE now

In [ ]:
# Step 1: Calculate the overall mean of each feature
FeatureMean <- colMeans(ft_t) # Getting Feature-centric mean

In [ ]:
# Step 2: Calculate the overall mean of each batch
new_md$ATTRIBUTE_Batch <- as.factor(new_md$ATTRIBUTE_Batch) #converting to factor type

In [ ]:
ft_md <- full_join(new_md,ft_t,by=c("filename"))
head(ft_md,n=3)

In [ ]:
B1_Mean <- ft_md %>% filter(`ATTRIBUTE_Batch` == 1) %>% summarize(across(-c(1:14), sum))
B2_Mean <- ft_md %>% filter(`ATTRIBUTE_Batch` == 2) %>% summarize(across(-c(1:14), sum))
B3_Mean <- ft_md %>% filter(`ATTRIBUTE_Batch` == 3) %>% summarize(across(-c(1:14), sum))

In [ ]:
#Step 3:
for (i in 1:nrow(Batch_1)){
  Batch_1[i,] <- (Batch_1[i,]/(B1_Mean+1))*(FeatureMean+1) #added 1 on numerator & denominator
}

for (i in 1:nrow(Batch_2)){
  Batch_2[i,] <- (Batch_2[i,]/(B2_Mean+1))*(FeatureMean+1)
}

for (i in 1:nrow(Batch_3)){
  Batch_3[i,] <- (Batch_3[i,]/(B3_Mean+1))*(FeatureMean+1)
}

In [ ]:
# I need to reproduce the batch mean like this:
Batch1_md <- new_md %>% filter(`ATTRIBUTE_Batch` == 1)
Batch_1 <- ft_t[rownames(Batch1_md),,drop=F]
B1_Mean <- colMeans(Batch_1)

We have 3 batches in our data

In [ ]:
Final <- as.data.frame(rbind(Batch_1,Batch_2,Batch_3))

### Getting the PCoA plot after Batch correction

In [ ]:
Final_S <-scale(Final, scale = T, center = F) #scaling the table
identical(rownames(Final_S),rownames(new_md))
Final_S <- Final_S[order(rownames(Final_S)),,drop=F] #ordering by rownames
identical(rownames(Final_S),rownames(new_md))

In [ ]:
distm <- vegdist(Final_S, method = "bray")# compute distance
PcoA <- cmdscale(distm, k = 2, eig = T, add = T)
PcoA_points <- as.data.frame(PcoA$points)
variance <- round(PcoA$eig*100/sum(PcoA$eig),1)
names(PcoA_points)[1:2] <- c('PCoA1', 'PCoA2')
identical(rownames(PcoA_points), rownames(new_md))

In [ ]:
ggplot(PcoA_points, aes(x = PCoA1, y = PCoA2, colour = as.factor(new_md$ATTRIBUTE_Batch), label = row.names(PcoA))) +
  geom_point(size=2, alpha=0.6) +
  scale_colour_manual(values = c('orange','darkgreen','red','blue','black')) +
  xlab(paste('PCoA1: ',variance[1],'%', sep = ' ')) +
  ylab(paste('PCoA2: ',variance[2],'%', sep = ' ')) +
  ggtitle("PCoA Scores plot on Batch corrected data using Bray-Curtis Dissimilarity Index")+
  theme(legend.title=element_blank())